In [1]:
import numpy as np
import pandas as pd
import requests
import tempfile
import os
import pyreadstat

#------------------------------------------------------
# 1. Loader that downloads any NHANES .xpt file safely
#------------------------------------------------------
def load_nhanes_xpt(file, year="2015"):
    url = f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{year}/DataFiles/{file}"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0 Safari/537.36"
        )
    }
    r = requests.get(url, headers=headers)
    r.raise_for_status()

    if b"<html" in r.content[:200].lower():
        preview = r.content[:500].decode(errors="ignore")
        raise ValueError(f"HTML returned instead of XPT:\n{url}\n\nPreview:\n{preview}")

    with tempfile.NamedTemporaryFile(suffix=".xpt", delete=False) as tmp:
        tmp.write(r.content)
        tmp_path = tmp.name

    try:
        df, meta = pyreadstat.read_xport(tmp_path)
    finally:
        os.remove(tmp_path)

    return df


In [2]:
nhanes_files = {
    "DEMO":      "DEMO_I.xpt",
    "HDL":       "HDL_I.xpt",
    "TCHOL":     "TCHOL_I.xpt",
    "TRIGLY":    "TRIGLY_I.xpt",
    "GLU":       "GLU_I.xpt",
    "INS":       "INS_I.xpt",
    "DPQ":       "DPQ_I.xpt",
    "SLQ":       "SLQ_I.xpt",
    "DR1TOT":    "DR1TOT_I.xpt",
    "DR1IFF":    "DR1IFF_I.xpt",  # <-- we will EXCLUDE this later
    "PAQ":       "PAQ_I.xpt",
    "BPX":       "BPX_I.xpt",
    "BIOPRO":    "BIOPRO_I.xpt",
    "ALB_CR":    "ALB_CR_I.xpt"
}


In [3]:
all_dfs = {}

for name, fname in nhanes_files.items():
    print(f"Downloading {name} ...")
    d = load_nhanes_xpt(fname, "2015")
    print(name, d.shape)
    all_dfs[name] = d


DEMO (9971, 47)
HDL (8021, 3)
TCHOL (8021, 3)
TRIGLY (3191, 6)
GLU (3191, 4)
INS (3191, 7)
DPQ (5735, 11)
SLQ (6327, 8)
DR1TOT (9544, 168)
DR1IFF (121481, 84)
PAQ (9255, 94)
BPX (9544, 21)
BIOPRO (6744, 38)
ALB_CR (8608, 8)


In [4]:
fped, meta = pyreadstat.read_sas7bdat("fped_dr1tot_1516.sas7bdat")
print("FPED shape:", fped.shape)

FPED shape: (9544, 51)


In [5]:
print("Does merged contain DEMO columns?")
print("RIAGENDR" in merged.columns)
print("RIDAGEYR" in merged.columns)
print("RIDRETH3" in merged.columns)
print("Number of columns:", len(merged.columns))

merged.columns.tolist()[:50]  # preview first 50 cols


Does merged contain DEMO columns?


NameError: name 'merged' is not defined

In [ ]:
# ================================================================
#  FULL NHANES → FEATURE ENGINEERING → rPDQS → LONGEVITY 1.1
# ================================================================

import numpy as np
import pandas as pd

# ------------------------------------------------------------
# 0. Start from merged NHANES + FPED person-level
# ------------------------------------------------------------
df = merged.copy()

# Merge FPED (person-level DR1TOT) onto merged NHANES
df = df.merge(fped, on="SEQN", how="left")


# ============================================================
# 1. BASIC DEMOGRAPHICS
# ============================================================
df["sex"] = df["RIAGENDR"]                     # 1=Male, 2=Female
df["race_ethnicity"] = df["RIDRETH3"]
df["age"] = df["RIDEXAGM"] / 12                # NHANES reports age in months
df["poverty_income_ratio"] = df["INDFMIN2"]


# ============================================================
# 2. RENAME LAB VARIABLES
# ============================================================
rename_map = {
    "LBXGLU": "fasting_glucose",
    "LBXIN": "fasting_insulin",
    "LBDHDD": "hdl_cholesterol",
    "LBDHDL": "hdl_cholesterol",
    "LBDTCSI": "total_cholesterol",
    "LBDTRSI": "triglycerides",
}
df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns}, inplace=True)


# ============================================================
# 3. LDL + ApoB
# ============================================================
def friedewald(row):
    tg = row["triglycerides"]
    if pd.isna(tg) or tg >= 400:
        return np.nan
    return row["total_cholesterol"] - row["hdl_cholesterol"] - tg/5

df["ldl_cholesterol"] = df.apply(friedewald, axis=1)
df["apob_est"] = 0.65 * df["ldl_cholesterol"] + 0.1 * df["triglycerides"]


# ============================================================
# 4. HOMA-IR
# ============================================================
df["homa_ir"] = (df["fasting_insulin"] * df["fasting_glucose"]) / 405


# ============================================================
# 5. BLOOD PRESSURE
# ============================================================
sbp_cols = [c for c in df.columns if c.startswith("BPXSY")]
dbp_cols = [c for c in df.columns if c.startswith("BPXDI")]

df["sbp"] = df[sbp_cols].mean(axis=1)
df["dbp"] = df[dbp_cols].mean(axis=1)
df["pulse_pressure"] = df["sbp"] - df["dbp"]


# ============================================================
# 6. eGFR — CKD-EPI 2021
# ============================================================
if "LBXSCR" in df.columns:
    scr = df["LBXSCR"]
    k = np.where(df["sex"] == 2, 0.7, 0.9)
    alpha = np.where(df["sex"] == 2, -0.241, -0.302)

    min_part = np.minimum(scr / k, 1) ** alpha
    max_part = np.maximum(scr / k, 1) ** -1.2

    df["egfr"] = 142 * min_part * max_part * (0.9938 ** df["age"])
else:
    df["egfr"] = np.nan


# ============================================================
# 7. SLEEP — SLQ030
# ============================================================
df["sleep_hours"] = df["SLQ030"]

def calc_sleep_score(x):
    if pd.isna(x): return np.nan
    if 7 <= x <= 9: return 100
    if 6 <= x < 7 or 9 < x <= 10: return 80
    return 50

df["sleep_score"] = df["sleep_hours"].apply(calc_sleep_score)


# ============================================================
# 8. PHQ-9
# ============================================================
phq_cols = [c for c in df.columns if c.startswith("DPQ0")]
df["phq9_total"] = df[phq_cols].sum(axis=1)


# ============================================================
# 9. MVPA — leisure activity
# ============================================================
if all(c in df.columns for c in ["PAD615","PAD630","PAD645","PAD660"]):
    vig = df["PAD615"] * df["PAD630"]
    mod = df["PAD645"] * df["PAD660"]
    df["mvpa_min_week"] = vig * 2 + mod
else:
    df["mvpa_min_week"] = np.nan


# ============================================================
# 10. rPDQS DIET QUALITY
# ============================================================

# --- Restrict to reliable day-1 recall
if "DR1DRSTZ" in df.columns:
    mask_bad = df["DR1DRSTZ"] != 1
else:
    mask_bad = pd.Series(False, index=df.index)

# Use FPED variables directly
servings = df[[
    "SEQN",
    "DR1T_V_DRKGR","DR1T_V_OTHER","DR1T_V_REDOR_TOMATO",
    "DR1T_V_LEGUMES","DR1T_G_WHOLE","DR1T_G_REFINED",
    "DR1T_F_OTHER","DR1T_F_CITMLB","DR1T_PF_NUTSDS",
    "DR1T_PF_MEAT","DR1T_PF_CUREDMEAT",
    "DR1T_PF_SEAFD_HI","DR1T_PF_SEAFD_LOW",
    "DR1T_D_MILK","DR1T_D_YOGURT","DR1T_A_DRINKS",
    "DR1T_SOLID_FATS"
]].copy()

# Zero out unreliable recalls
servings.loc[mask_bad, servings.columns[1:]] = np.nan
servings.set_index("SEQN", inplace=True)

# Build food groups
servings["dark_green_veg"] = servings["DR1T_V_DRKGR"]
servings["other_veg"] = (
    servings["DR1T_V_OTHER"] +
    servings["DR1T_V_REDOR_TOMATO"]
    - servings["DR1T_V_LEGUMES"].fillna(0)
)
servings["citrus_melons_berries"] = servings["DR1T_F_CITMLB"]
servings["other_fruit"] = servings["DR1T_F_OTHER"]
servings["legumes"] = servings["DR1T_V_LEGUMES"]
servings["whole_grains"] = servings["DR1T_G_WHOLE"]
servings["nuts_seeds"] = servings["DR1T_PF_NUTSDS"]
servings["low_fat_dairy"] = servings["DR1T_D_MILK"] + servings["DR1T_D_YOGURT"]
servings["fish"] = servings["DR1T_PF_SEAFD_HI"] + servings["DR1T_PF_SEAFD_LOW"]

servings["red_meat"] = servings["DR1T_PF_MEAT"]
servings["processed_meat"] = servings["DR1T_PF_CUREDMEAT"]
servings["refined_grains"] = servings["DR1T_G_REFINED"]
servings["ssb"] = servings["DR1T_A_DRINKS"]
servings["fried_foods"] = servings["DR1T_SOLID_FATS"]

healthy = ["dark_green_veg","other_veg","citrus_melons_berries","other_fruit",
           "legumes","whole_grains","nuts_seeds","low_fat_dairy","fish"]
unhealthy = ["red_meat","processed_meat","refined_grains","ssb","fried_foods"]

# Scoring functions
def pos(s): return pd.qcut(s.rank(method="first"), 5, labels=[0,1,2,3,4], duplicates="drop").astype(float)
def neg(s): return pd.qcut(s.rank(method="first"), 5, labels=[4,3,2,1,0], duplicates="drop").astype(float)

for g in healthy:
    servings[g + "_score"] = pos(servings[g])

for g in unhealthy:
    servings[g + "_score"] = neg(servings[g])

score_cols = [g + "_score" for g in healthy + unhealthy]
servings["rpdqs_total"] = servings[score_cols].sum(axis=1)
servings["rpdqs_normalized"] = (servings["rpdqs_total"] / 56) * 100

df = df.merge(servings[["rpdqs_total","rpdqs_normalized"]],
              left_on="SEQN", right_index=True, how="left")


# ============================================================
# SAVE FEATURE-ENGINEERED DATASET
# ============================================================
df_final = df.copy()

print("df_final shape:", df_final.shape)
print(df_final[["SEQN","rpdqs_normalized"]].head())


# ============================================================
# 11. LONGEVITY SCORE v1.1 NORMALIZATION
# ============================================================

df = df_final.copy()

def normalize_positive(x, low, high):
    return ((x - low) / (high - low)) * 100

def normalize_negative(x, low, high):
    return ((high - x) / (high - low)) * 100

# BMI
if {"BMXWT","BMXHT"}.issubset(df.columns):
    df["bmi"] = df["BMXWT"]/(df["BMXHT"]/100)**2
df["bmi_norm"] = normalize_negative(df["bmi"],18.5,35)

# ApoB
df["apob_norm"] = normalize_negative(df["apob_est"],40,120)

# CRP
df["crp_norm"] = normalize_negative(df["LBXCRP"],0.1,10)

# PHQ-9
df["phq9_norm"] = normalize_negative(df["phq9_total"],0,27)

# ALT
df["alt_norm"] = normalize_negative(df["LBXSGPT"],8,50)

# eGFR
df["egfr_norm"] = normalize_positive(df["egfr"],60,110)

# MVPA
df["mvpa_norm"] = normalize_positive(df["mvpa_min_week"],0,300)

# Diet
df["diet_norm"] = df["rpdqs_normalized"]

# All other fields = NaN place-holders
df["ogtt_norm"]=np.nan
df["vo2max_norm"]=np.nan
df["pack_years_norm"]=np.nan
df["moca_norm"]=np.nan
df["cac_norm"]=np.nan
df["hrv_norm"]=np.nan
df["bmd_norm"]=np.nan
df["truage_norm"]=np.nan
df["shdl_norm"]=np.nan
df["rem_norm"]=np.nan
df["grip_norm"]=np.nan
df["swls_norm"]=np.nan

# Final 20 variables
longevity_inputs=[
"ogtt_norm","apob_norm","vo2max_norm","crp_norm","bmi_norm",
"pack_years_norm","moca_norm","mvpa_norm","cac_norm","hrv_norm",
"phq9_norm","alt_norm","egfr_norm","bmd_norm","truage_norm",
"shdl_norm","rem_norm","grip_norm","swls_norm","diet_norm"
]

df_longevity = df[["SEQN"]+longevity_inputs]
print(df_longevity.head())


In [ ]:
# Start from DEMO (person-level gold standard)
df = all_dfs["DEMO"].copy()

# merge all other person-level NHANES datasets except DR1IFF
for name, d in all_dfs.items():
    if name in ["DEMO", "DR1IFF"]:   # exclude DEMO (already loaded) and DR1IFF (food item file)
        continue
    df = df.merge(d, on="SEQN", how="left")

# merge FPED food-group data
df = df.merge(fped, on="SEQN", how="left")

print("Merged person-level shape:", df.shape)
df.head()

In [ ]:
# ---------------------------
# SAFELY assign sex and age
# ---------------------------

# --- SEX ---
if "sex" in df.columns:
    # already present (from rename or earlier merge)
    df["sex"] = df["sex"]
elif "RIAGENDR" in df.columns:
    df["sex"] = df["RIAGENDR"]
elif "RIAGENDER" in df.columns:
    df["sex"] = df["RIAGENDER"]
elif "RIAGENDR_x" in df.columns:
    df["sex"] = df["RIAGENDR_x"]
elif "RIAGENDR_y" in df.columns:
    df["sex"] = df["RIAGENDR_y"]
else:
    print("⚠️ WARNING: No sex variable found. Assigning NaN.")
    df["sex"] = np.nan

# --- AGE ---
if "age" in df.columns:
    df["age"] = df["age"]
elif "RIDAGEYR" in df.columns:
    df["age"] = df["RIDAGEYR"]
elif "RIDAGEYR_x" in df.columns:
    df["age"] = df["RIDAGEYR_x"]
elif "RIDAGEYR_y" in df.columns:
    df["age"] = df["RIDAGEYR_y"]
elif "RIDAGEY" in df.columns:  # some cycles
    df["age"] = df["RIDAGEY"]
else:
    print("⚠️ WARNING: No age variable found. Assigning NaN.")
    df["age"] = np.nan


In [ ]:
# ================================================================
#  FULL NHANES → FEATURE ENGINEERING → rPDQS → LONGEVITY 1.1
# ================================================================

import numpy as np
import pandas as pd

# ------------------------------------------------------------
# 0. Start from merged NHANES + FPED person-level
# ------------------------------------------------------------
df = merged.copy()

# Merge FPED (person-level DR1TOT) onto merged NHANES
df = df.merge(fped, on="SEQN", how="left")


# ============================================================
# 1. BASIC DEMOGRAPHICS
# ============================================================
df["sex"] = df["RIAGENDR"]                     # 1=Male, 2=Female
df["race_ethnicity"] = df["RIDRETH3"]
df["age"] = df["RIDEXAGM"] / 12                # NHANES reports age in months
df["poverty_income_ratio"] = df["INDFMIN2"]


# ============================================================
# 2. RENAME LAB VARIABLES
# ============================================================
rename_map = {
    "LBXGLU": "fasting_glucose",
    "LBXIN": "fasting_insulin",
    "LBDHDD": "hdl_cholesterol",
    "LBDHDL": "hdl_cholesterol",
    "LBDTCSI": "total_cholesterol",
    "LBDTRSI": "triglycerides",
}
df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns}, inplace=True)


# ============================================================
# 3. LDL + ApoB
# ============================================================
def friedewald(row):
    tg = row["triglycerides"]
    if pd.isna(tg) or tg >= 400:
        return np.nan
    return row["total_cholesterol"] - row["hdl_cholesterol"] - tg/5

df["ldl_cholesterol"] = df.apply(friedewald, axis=1)
df["apob_est"] = 0.65 * df["ldl_cholesterol"] + 0.1 * df["triglycerides"]


# ============================================================
# 4. HOMA-IR
# ============================================================
df["homa_ir"] = (df["fasting_insulin"] * df["fasting_glucose"]) / 405


# ============================================================
# 5. BLOOD PRESSURE
# ============================================================
sbp_cols = [c for c in df.columns if c.startswith("BPXSY")]
dbp_cols = [c for c in df.columns if c.startswith("BPXDI")]

df["sbp"] = df[sbp_cols].mean(axis=1)
df["dbp"] = df[dbp_cols].mean(axis=1)
df["pulse_pressure"] = df["sbp"] - df["dbp"]


# ============================================================
# 6. eGFR — CKD-EPI 2021
# ============================================================
if "LBXSCR" in df.columns:
    scr = df["LBXSCR"]
    k = np.where(df["sex"] == 2, 0.7, 0.9)
    alpha = np.where(df["sex"] == 2, -0.241, -0.302)

    min_part = np.minimum(scr / k, 1) ** alpha
    max_part = np.maximum(scr / k, 1) ** -1.2

    df["egfr"] = 142 * min_part * max_part * (0.9938 ** df["age"])
else:
    df["egfr"] = np.nan


# ============================================================
# 7. SLEEP — SLQ030
# ============================================================
df["sleep_hours"] = df["SLQ030"]

def calc_sleep_score(x):
    if pd.isna(x): return np.nan
    if 7 <= x <= 9: return 100
    if 6 <= x < 7 or 9 < x <= 10: return 80
    return 50

df["sleep_score"] = df["sleep_hours"].apply(calc_sleep_score)


# ============================================================
# 8. PHQ-9
# ============================================================
phq_cols = [c for c in df.columns if c.startswith("DPQ0")]
df["phq9_total"] = df[phq_cols].sum(axis=1)


# ============================================================
# 9. MVPA — leisure activity
# ============================================================
if all(c in df.columns for c in ["PAD615","PAD630","PAD645","PAD660"]):
    vig = df["PAD615"] * df["PAD630"]
    mod = df["PAD645"] * df["PAD660"]
    df["mvpa_min_week"] = vig * 2 + mod
else:
    df["mvpa_min_week"] = np.nan


# ============================================================
# 10. rPDQS DIET QUALITY
# ============================================================

# --- Restrict to reliable day-1 recall
if "DR1DRSTZ" in df.columns:
    mask_bad = df["DR1DRSTZ"] != 1
else:
    mask_bad = pd.Series(False, index=df.index)

# Use FPED variables directly
servings = df[[
    "SEQN",
    "DR1T_V_DRKGR","DR1T_V_OTHER","DR1T_V_REDOR_TOMATO",
    "DR1T_V_LEGUMES","DR1T_G_WHOLE","DR1T_G_REFINED",
    "DR1T_F_OTHER","DR1T_F_CITMLB","DR1T_PF_NUTSDS",
    "DR1T_PF_MEAT","DR1T_PF_CUREDMEAT",
    "DR1T_PF_SEAFD_HI","DR1T_PF_SEAFD_LOW",
    "DR1T_D_MILK","DR1T_D_YOGURT","DR1T_A_DRINKS",
    "DR1T_SOLID_FATS"
]].copy()

# Zero out unreliable recalls
servings.loc[mask_bad, servings.columns[1:]] = np.nan
servings.set_index("SEQN", inplace=True)

# Build food groups
servings["dark_green_veg"] = servings["DR1T_V_DRKGR"]
servings["other_veg"] = (
    servings["DR1T_V_OTHER"] +
    servings["DR1T_V_REDOR_TOMATO"]
    - servings["DR1T_V_LEGUMES"].fillna(0)
)
servings["citrus_melons_berries"] = servings["DR1T_F_CITMLB"]
servings["other_fruit"] = servings["DR1T_F_OTHER"]
servings["legumes"] = servings["DR1T_V_LEGUMES"]
servings["whole_grains"] = servings["DR1T_G_WHOLE"]
servings["nuts_seeds"] = servings["DR1T_PF_NUTSDS"]
servings["low_fat_dairy"] = servings["DR1T_D_MILK"] + servings["DR1T_D_YOGURT"]
servings["fish"] = servings["DR1T_PF_SEAFD_HI"] + servings["DR1T_PF_SEAFD_LOW"]

servings["red_meat"] = servings["DR1T_PF_MEAT"]
servings["processed_meat"] = servings["DR1T_PF_CUREDMEAT"]
servings["refined_grains"] = servings["DR1T_G_REFINED"]
servings["ssb"] = servings["DR1T_A_DRINKS"]
servings["fried_foods"] = servings["DR1T_SOLID_FATS"]

healthy = ["dark_green_veg","other_veg","citrus_melons_berries","other_fruit",
           "legumes","whole_grains","nuts_seeds","low_fat_dairy","fish"]
unhealthy = ["red_meat","processed_meat","refined_grains","ssb","fried_foods"]

# Scoring functions
def pos(s): return pd.qcut(s.rank(method="first"), 5, labels=[0,1,2,3,4], duplicates="drop").astype(float)
def neg(s): return pd.qcut(s.rank(method="first"), 5, labels=[4,3,2,1,0], duplicates="drop").astype(float)

for g in healthy:
    servings[g + "_score"] = pos(servings[g])

for g in unhealthy:
    servings[g + "_score"] = neg(servings[g])

score_cols = [g + "_score" for g in healthy + unhealthy]
servings["rpdqs_total"] = servings[score_cols].sum(axis=1)
servings["rpdqs_normalized"] = (servings["rpdqs_total"] / 56) * 100

df = df.merge(servings[["rpdqs_total","rpdqs_normalized"]],
              left_on="SEQN", right_index=True, how="left")


# ============================================================
# SAVE FEATURE-ENGINEERED DATASET
# ============================================================
df_final = df.copy()

print("df_final shape:", df_final.shape)
print(df_final[["SEQN","rpdqs_normalized"]].head())


# ============================================================
# 11. LONGEVITY SCORE v1.1 NORMALIZATION
# ============================================================

df = df_final.copy()

def normalize_positive(x, low, high):
    return ((x - low) / (high - low)) * 100

def normalize_negative(x, low, high):
    return ((high - x) / (high - low)) * 100

# BMI
if {"BMXWT","BMXHT"}.issubset(df.columns):
    df["bmi"] = df["BMXWT"]/(df["BMXHT"]/100)**2
df["bmi_norm"] = normalize_negative(df["bmi"],18.5,35)

# ApoB
df["apob_norm"] = normalize_negative(df["apob_est"],40,120)

# CRP
df["crp_norm"] = normalize_negative(df["LBXCRP"],0.1,10)

# PHQ-9
df["phq9_norm"] = normalize_negative(df["phq9_total"],0,27)

# ALT
df["alt_norm"] = normalize_negative(df["LBXSGPT"],8,50)

# eGFR
df["egfr_norm"] = normalize_positive(df["egfr"],60,110)

# MVPA
df["mvpa_norm"] = normalize_positive(df["mvpa_min_week"],0,300)

# Diet
df["diet_norm"] = df["rpdqs_normalized"]

# All other fields = NaN place-holders
df["ogtt_norm"]=np.nan
df["vo2max_norm"]=np.nan
df["pack_years_norm"]=np.nan
df["moca_norm"]=np.nan
df["cac_norm"]=np.nan
df["hrv_norm"]=np.nan
df["bmd_norm"]=np.nan
df["truage_norm"]=np.nan
df["shdl_norm"]=np.nan
df["rem_norm"]=np.nan
df["grip_norm"]=np.nan
df["swls_norm"]=np.nan

# Final 20 variables
longevity_inputs=[
"ogtt_norm","apob_norm","vo2max_norm","crp_norm","bmi_norm",
"pack_years_norm","moca_norm","mvpa_norm","cac_norm","hrv_norm",
"phq9_norm","alt_norm","egfr_norm","bmd_norm","truage_norm",
"shdl_norm","rem_norm","grip_norm","swls_norm","diet_norm"
]

df_longevity = df[["SEQN"]+longevity_inputs]
print(df_longevity.head())



# feature engineering

# build rPDQS score

In [ ]:
fped.columns[:20]



In [ ]:
df["DR1DRSTZ"].value_counts(dropna=False).sort_index()


In [ ]:
import numpy as np
import pandas as pd

# ==========================================================
# 1. Extract DR1IFF (food-item level file)
# ==========================================================
dr1 = all_dfs["DR1IFF"].copy()

# Keep only necessary columns
needed_cols = ["SEQN", "DR1IFDCD", "DR1IGRMS"]
missing_cols = [c for c in needed_cols if c not in dr1.columns]
if missing_cols:
    raise ValueError(f"Missing columns required for rPDQS: {missing_cols}")

dr1 = dr1[needed_cols].copy()

# ==========================================================
# 2. Create the FPED 4-digit food-group code
# ==========================================================
dr1["fg_code"] = (dr1["DR1IFDCD"] // 10000).astype("Int64")

# ==========================================================
# 3. Official rPDQS food-group map
# ==========================================================
rpdqs_map = {
    "dark_green_veg":       [6310, 6320],
    "other_veg":            [6110, 6120, 6130, 6140],
    "citrus_melons_berries":[6210, 6220, 6230],
    "other_fruit":          [6240, 6250],
    "legumes":              [7510],
    "whole_grains":         [5710, 5720],
    "nuts_seeds":           [7410, 7420],
    "low_fat_dairy":        [1310, 1320],
    "fish":                 [2710, 2720],

    # Negative food groups
    "red_meat":             [2510, 2520],
    "processed_meat":       [2530, 2540],
    "refined_grains":       [5610, 5620],
    "ssb":                  [9310, 9320, 9330],
    "fried_foods":          [6410, 6420]
}

# ==========================================================
# 4. Map each fg_code → rPDQS category
# ==========================================================
reverse_map = {code: group for group, codes in rpdqs_map.items() for code in codes}

def classify_food(code):
    return reverse_map.get(code, None)

dr1["rpdqs_cat"] = dr1["fg_code"].apply(classify_food)

# Drop rows not part of rPDQS
dr1 = dr1.dropna(subset=["rpdqs_cat"])

# ==========================================================
# 5. Compute total grams per category per SEQN
# ==========================================================
servings = (
    dr1.groupby(["SEQN", "rpdqs_cat"])["DR1IGRMS"]
       .sum()
       .unstack()
       .fillna(0)
)

# ==========================================================
# 6. Safe quintile scoring function
# ==========================================================
def safe_qcut(series, labels):
    s = series.fillna(0)

    try:
        q = pd.qcut(s, 5, labels=labels, duplicates="drop")
        # if bins collapse (common), fallback:
        if len(q.cat.categories) != len(labels):
            return pd.Series([labels[0]] * len(s), index=s.index, dtype=float)
        return q.astype(float)
    except Exception:
        # if all zeros or qcut cannot run:
        return pd.Series([labels[0]] * len(s), index=s.index, dtype=float)

# ==========================================================
# 7. Apply scoring rules
# ==========================================================
positive = [
    "dark_green_veg","other_veg","citrus_melons_berries","other_fruit",
    "legumes","whole_grains","nuts_seeds","low_fat_dairy","fish"
]

negative = ["red_meat","processed_meat","refined_grains","ssb","fried_foods"]

# Positive groups: 0 → 4
for col in positive:
    if col in servings.columns:
        servings[col + "_score"] = safe_qcut(servings[col], labels=[0,1,2,3,4])
    else:
        servings[col + "_score"] = 0

# Negative groups: 4 → 0
for col in negative:
    if col in servings.columns:
        servings[col + "_score"] = safe_qcut(servings[col], labels=[4,3,2,1,0])
    else:
        servings[col + "_score"] = 0

# ==========================================================
# 8. Total rPDQS and normalized score (0–100)
# ==========================================================
score_cols = [c for c in servings.columns if c.endswith("_score")]

servings["rpdqs_total"] = servings[score_cols].sum(axis=1)

servings["rpdqs_normalized"] = (
    servings["rpdqs_total"] / (len(score_cols) * 4) * 100
)
# Remove any previous rPDQS columns before re-merging
df = df.drop(columns=[c for c in df.columns if c in ["rpdqs_total","rpdqs_normalized"]], errors="ignore")

# Merge rPDQS results back into df
df = df.merge(
    servings[["rpdqs_total", "rpdqs_normalized"]],
    left_on="SEQN", right_index=True, how="left"
)

print("✓ rPDQS columns now present:",
      "rpdqs_total" in df.columns,
      "rpdqs_normalized" in df.columns)

print(df[["SEQN","rpdqs_total","rpdqs_normalized"]].head())

## build BMI and normalize variables

In [ ]:
# ----------------------------------------------------------
# Longevity Score v1.1: Compute & Normalize all components
# ----------------------------------------------------------

df = df_final.copy()



# -----------------------
# 1. BMI
# -----------------------
# NHANES variables:
# BMXWT = weight in kg
# BMXHT = height in cm
if {"BMXWT", "BMXHT"}.issubset(df.columns):
    df["bmi"] = df["BMXWT"] / (df["BMXHT"] / 100)**2
else:
    df["bmi"] = np.nan


# -----------------------
# 2. Normalize key domains
#    Each normalization maps raw values → 0–100 scale
# -----------------------

def normalize_positive(x, low, high):
    """Higher is better."""
    return ((x - low) / (high - low)) * 100

def normalize_negative(x, low, high):
    """Lower is better (e.g., CRP, ApoB, BMI)."""
    return ((high - x) / (high - low)) * 100


# -----------------------
# VO2max — NHANES DOES NOT HAVE
# Placeholder = NaN
# -----------------------
df["vo2max_raw"] = np.nan
df["vo2max_norm"] = np.nan


# -----------------------
# 2-hour OGTT — not in NHANES
# -----------------------
df["ogtt_raw"] = np.nan
df["ogtt_norm"] = np.nan


# -----------------------
# ApoB (estimated)
# -----------------------
if "apob_est" in df.columns:
    df["apob_norm"] = normalize_negative(df["apob_est"], low=40, high=120)
else:
    df["apob_norm"] = np.nan


# -----------------------
# CRP (NHANES variable: LBXCRP)
# -----------------------
if "LBXCRP" in df.columns:
    df["crp_norm"] = normalize_negative(df["LBXCRP"], low=0.1, high=10)
else:
    df["crp_norm"] = np.nan


# -----------------------
# BMI
# -----------------------
df["bmi_norm"] = normalize_negative(df["bmi"], low=18.5, high=35)


# -----------------------
# Smoking (pack-years)
# NHANES has: SMQ020 (ever smoked), SMD641 (years smoked)
# We estimate pack-years crudely.
# -----------------------
if {"SMD630", "SMD641"}.issubset(df.columns):
    df["cigs_per_day"] = df["SMD630"]
    df["years_smoked"] = df["SMD641"]
    df["pack_years"] = (df["cigs_per_day"] / 20) * df["years_smoked"]
    df["pack_years_norm"] = normalize_negative(df["pack_years"], low=0, high=50)
else:
    df["pack_years_norm"] = np.nan


# -----------------------
# Cognitive function — MoCA not in NHANES
# -----------------------
df["moca_norm"] = np.nan


# -----------------------
# MVPA (already computed)
# -----------------------
if "mvpa_min_week" in df.columns:
    df["mvpa_norm"] = normalize_positive(df["mvpa_min_week"], low=0, high=300)
else:
    df["mvpa_norm"] = np.nan


# -----------------------
# CAC — not in NHANES
# -----------------------
df["cac_norm"] = np.nan


# -----------------------
# HRV — not in NHANES
# -----------------------
df["hrv_norm"] = np.nan


# -----------------------
# PHQ-9 mental health
# -----------------------
df["phq9_norm"] = normalize_negative(df["phq9_total"], low=0, high=27)


# -----------------------
# ALT (liver enzyme)
# NHANES variable: LBXSGPT
# -----------------------
if "LBXSGPT" in df.columns:
    df["alt_norm"] = normalize_negative(df["LBXSGPT"], low=8, high=50)
else:
    df["alt_norm"] = np.nan


# -----------------------
# eGFR
# -----------------------
df["egfr_norm"] = normalize_positive(df["egfr"], low=60, high=110)


# -----------------------
# Bone Mineral Density — NHANES has DEXA data but not in your load
# -----------------------
df["bmd_norm"] = np.nan


# -----------------------
# Epigenetic Age (TruAge) — not in NHANES
# -----------------------
df["truage_norm"] = np.nan


# -----------------------
# Small HDL particles — not in NHANES
# -----------------------
df["shdl_norm"] = np.nan


# -----------------------
# REM sleep — not in NHANES
# -----------------------
df["rem_norm"] = np.nan


# -----------------------
# Grip strength — NHANES variable exists (MGDCGSZ)
# We will check and normalize
# -----------------------
grip_vars = [c for c in df.columns if "MGDC" in c]
if grip_vars:
    df["grip_strength"] = df[grip_vars].max(axis=1)
    df["grip_norm"] = normalize_positive(df["grip_strength"], low=10, high=60)
else:
    df["grip_norm"] = np.nan


# -----------------------
# Life Satisfaction (SWLS) — not in NHANES
# -----------------------
df["swls_norm"] = np.nan


# -----------------------
# Diet quality (rPDQS, already computed)
# -----------------------
df["diet_norm"] = df["rpdqs_normalized"]   # already 0–100


# -----------------------
# Final Longevity Score v1.1 inputs only
# -----------------------
longevity_inputs = [
    "ogtt_norm","apob_norm","vo2max_norm","crp_norm","bmi_norm",
    "pack_years_norm","moca_norm","mvpa_norm","cac_norm","hrv_norm",
    "phq9_norm","alt_norm","egfr_norm","bmd_norm","truage_norm",
    "shdl_norm","rem_norm","grip_norm","swls_norm","diet_norm"
]

df_longevity = df[["SEQN"] + longevity_inputs]

print("Longevity variable matrix shape:", df_longevity.shape)
df_longevity.head()


## exploratory data analysis

In [ ]:
df_final.shape

In [ ]:
df_final.info()



In [ ]:
df_final.head()


In [ ]:
df_final.describe(include='all').T

In [ ]:
# why are some rPDQS missing?

df_final.loc[df_final['rpdqs_cat'].isna(), ['SEQN','DR1DRSTZ','RIDAGEYR','WTDR2D']]


In [ ]:
df_final.columns


In [ ]:
df.columns


In [ ]:
df_rpdqs.columns



In [ ]:
import pandas as pd
[df_name for df_name in dir() if isinstance(eval(df_name), pd.DataFrame)]


In [ ]:
fped.columns


In [ ]:
merged.columns


In [ ]:
serving_table.columns


In [ ]:
df_final['SEQN'].isin(fped['SEQN']).mean()


In [ ]:
fped['DR1DRSTZ'].value_counts(dropna=False)


In [ ]:
df_final[~df_final['SEQN'].isin(fped['SEQN'])]['SEQN'].head()


In [ ]:
fped.loc[fped['SEQN'].isin([83758,83771,83782,83846,83850])]


In [ ]:
df_final.loc[df_final['SEQN'].isin([83758,83771,83782,83846,83850]),
             ['SEQN','rpdqs_total','rpdqs_cat']]


In [ ]:
fped[fped['SEQN'].isin([83758,83771,83782,83846,83850])]


In [ ]:
rpdqs_map


In [ ]:
seqns = [83758, 83771, 83782, 83846, 83850]

fped.loc[fped['SEQN'].isin(seqns)]


In [ ]:
for obj in dir():
    if "map" in obj.lower():
        print(obj)


In [ ]:
rename_map

In [ ]:
rpdqs_map

In [ ]:
import pandas as pd

[x for x in dir() if isinstance(eval(x), dict)]


In [ ]:
import re

for i, cell in enumerate(In):
    if re.search(r'rpdqs_cat', cell, flags=re.IGNORECASE):
        print(f"Cell {i}:")
        print(cell)
        print("-----")
